## MNIST 

### Introduction

In this project, I am going to implement a Deep Neural Network using TensorFlow 2.0 in a dataset called MNIST, which is about handwritten digit recognition. This is a classification problem.

The dataset I am going to use is the one provided by TensorFlow and consist of 70,000 images with 28x28 pixels of handwritten numbers. That means, each image corresponds to a number between 0 and 9.

My goal is to write an algorithm that identifies the handwritten number of the given images.

### Packages 

In [1]:
# import the relevant packages
import numpy as np
import tensorflow as tf

# the dataset is download the first time to my local 
# machine and the next time the code is run, the computer
# will just load the data from the stored folder
import tensorflow_datasets as tfds

### Data

In [2]:
# load the MNIST dataset for tensorflow dataset
# with_info=True to know version, size, etc. of the data 
# as_supervised=True return 2-tuple, input and target, structure
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
# we define the train and test set which are from default 
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [3]:
# there is no validation set, so we take a 10% from train set
num_validation_samples = 0.1*mnist_info.splits['train'].num_examples
# just in case the number of samples is not an integer
# we transform it to an integer
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

In [4]:
# also we create a variable with the number of test
# samples and we tranform from float to an integer
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

The dataset is normalised to have values between 0 and 1, instead of between 0 and 255. This is a common practice in ML, since it is desirable to have the same range of values for all the variables. This is done dividing by the maximum with a function and applying it to all the pixels in each image.

In [5]:
# the data is a grey scale from 0 to 255
# I define a function called scale to 
# translate it into a numbers from 0 to 1
def scale(image, label):
    # the value has to be a float to scale it
    image = tf.cast(image, tf.float32)
    image = image/255.
    
    return image, label

# application of the function for dataset
# first from the train that will be split 
# into train and validation
scaled_train_and_validation_data = mnist_train.map(scale)
# scale function applied to test dataset
test_data = mnist_test.map(scale)

Also, a common practice in ML is to shuffle the dataset. This is very useful practice to avoid local minimum for the loss function and to avoid ordered data that could confuse the neural network to make a wrong assumption.

In [6]:
# dealing with big datasets can make the computer run
# out of memory, therefore it is define a buffer_size 
# to shuffle by smaller groups of data
# 1 (no shuffle) < buffer_size <= num samples 
buffer_size = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(buffer_size)

Now is time to extract the validation data from the train dataset

In [7]:
# the validation data is taken from the train dataset
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
# the train data is the rest 
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

In [ ]:
batch_size = 150

train_data = train_data.batch(batch_size)

validation_data = validation_data.batch(num_validation_samples)

test_data = test_data.batch(num_test_samples)